This is part four of a multi part challenge.

Battleship is turn based. Please write a function that creates an `both yours and an opponents` board, the more random the board the better, with ships of the following types:

```
# Class of ship Size
1 Carrier          5
2 Battleship       4
2 Cruiser          3
2 Destroyer        2
```

Then allow playerOne and playerTwo to guess spaces, marking them either `e` or `h`

Your game should have functions similar in use case as these:
`createPlayerBoard('playerOne')`
`createPlayerBoard('playerTwo')`
`printMyBoard()`
`printOppenentBoard() // opponents board should not show where the ship is but should show past guesses.`

```
createPlayerBoard('playerOne')
createPlayerBoard('playerTwo')
printOppenentBoard()
guessLocation('4', 'h')

   1   2   3   4   5   6   7   8   9  10
a  u   u   u   u   u   u   u   u   u   u
b  u   e   u   u   u   e   u   u   u   u
c  u   u   u   u   u   u   u   u   u   u
e  u   u   e   u   u   u   u   u   u   u
f  u   u   u   u   u   u   u   u   u   u
g  u   u   u   u   u   u   u   u   u   u
h  u   u   u   e   u   h   h   u   u   u
i  u   u   u   u   u   u   u   u   u   u
j  u   u   u   u   u   u   u   u   u   u

```

In [ ]:
import string
import random
from enum import Enum

class InvalidPlacementException(Exception):
    pass

class Ship(Enum):
    CARRIER = 5
    BATTLESHIP = 4
    CRUISER = 3
    DESTROYER = 2

class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        elif isinstance(other, tuple):
            return Point(self.x + other[0], self.y + other[1])
        else:
            raise TypeError("Unsupported Type for addition {}".format(type(other)))
    
    def __mul__(self, other):
        if isinstance(other, Point):
            return Point(self.x * other.x, self.y * other.y)
        elif isinstance(other, tuple):
            return Point(self.x * other[0], self.y * other[1])
        elif isinstance(other, int):
            return Point(self.x * other, self.y * other)
        else:
            return TypeError("Unsupported Type of addition {}".format(type(other)))
                            
    def __repr__(self):
        return "Point<{0.x},{0.y}>".format(self)
    
    def to_tuple(self):
        return self.x, self.y
    
class Coordinate:
    def __init__(self, row: str, column: int):
        self.row = row
        self.column = int(column)
    
    def _letter_to_index(self, letter: str) -> int:
        return string.ascii_lowercase.index(letter)
    
    def __repr__(self):
        return "{0.row}{0.column}".format(self)
    
    def point(self):
        return Point(self.column - 1, self._letter_to_index(self.row))

class Board:
    directions = {
        "right": Point(1, 0),
        "down": Point(0, 1)
    }

    def __init__(self, n:int=10):
        self.size = n
        self.initial = 'u'
        self.boat = 'o'
        self.board = [[None for _ in range(self.size)] for _ in range(self.size)]

    def _get(self, point: Point):
        column, row = point.to_tuple()
        return self.board[row][column]
    
    def _set(self, point: Point, value):
        column, row = point.to_tuple()
        self.board[row][column] = value
        
    def placeBoat(self, ship: Ship, row: str, column: int, direction: str):
        start = Coordinate(row, column).point()
        d = self.directions[direction]
        spots = [start + (d * i) for i in range(0, ship.value)]
        ship
        try:
            if all(map(lambda x: self._get(x) is None, spots)):
                list(map(lambda x: self._set(x, self.boat), spots))
            else:
                raise InvalidPlacementException('Collides with another Boat!')
        except IndexError:
            raise InvalidPlacementException("Outside Game Bounds")
        
    def __repr__(self):
        display = [[n if n else " " for n in range(self.size + 1)]]
        for i in range(self.size):
            display.append([string.ascii_lowercase[i]] + self.board[i])
        return "\n".join(" ".join(map(lambda x: str(x).rjust(3) if x else self.initial.rjust(3), row)) for row in display)
    
class Game:
    board_size = 10
    ship_counts = {
        Ship.CARRIER: 1,
        Ship.BATTLESHIP: 2,
        Ship.CRUISER: 2,
        Ship.DESTROYER: 2
    }
    
    def __init__(self):
        self.player1 = Board(self.board_size)
        self.player2 = Board(self.board_size)
        
    def createOpponentBoard(self):
        for ship, count in self.ship_counts.items():
            for _ in range(count):
                while True:
                    try:
                        direction = random.choice(['right', 'down'])
                        row = string.ascii_lowercase[random.randrange(self.board_size)]
                        column = random.randrange(self.board_size)
                        self.player2.placeBoat(ship, row, column, direction)
                        break
                    except InvalidPlacementException:
                        pass
                
    def printOpponentBoard(self):
        print(self.player2)
        
    def printYourBoard(self):
        print(self.player1)
    
    def __repr__(self):
        return "\n".join([
          "Game:",
          "=" * 10,
          "Player1:",
          self.player1.__repr__(),
          "",
          "=" * 10,
          "Player2:",
          self.player2.__repr__()
        ])

if __name__ == "__main__":
    game = Game()
    game.createOpponentBoard()
    print(game)
